In [1]:
# The programing edited by Wade Wei Huang on 16 Jul 2021
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium.webdriver.common.keys import Keys
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
import sys
import chart_studio

In [2]:
# longin chart_studio
chart_studio.tools.set_credentials_file(username='whuang129', api_key='HosdO0DtPZ9gttx6aiJW')

In [2]:

# login website
chrome_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get('https://webap.sk88.com.tw/DESKTOP/#/')

# get elements
elem_user = driver.find_element_by_name("idno")
elem_user.clear
elem_user.send_keys(np.load("Keyinfo.npy")[0].split('%',2)[1])
# get elements
elem_pwd = driver.find_element_by_name("pass")
elem_pwd.clear
elem_pwd.send_keys(np.load("Keyinfo.npy")[0].split('%',2)[0])
# get elements
elem_button = driver.find_element_by_css_selector('div.col-xs-6.text-center')
webdriver.ActionChains(driver).click(elem_button).perform()
#
delay = 5 # seconds
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'IdOfMyElement')))
    print("Page is ready!")
except TimeoutException:
    print("Loading took too much time!")
    # driver.close()
# get elements
elem_close_button = driver.find_element_by_css_selector('div.modal-footer .btn.btn-default')
webdriver.ActionChains(driver).click(elem_close_button).perform()

# loading today data form the website
# get element
elem_menu = driver.find_element_by_xpath("//a[@class='dropdown-toggle']")
elem_menu.click()

elem_menu2 = driver.find_element_by_xpath("(//div[@class='col-xs-12'])[4]/ul/li[2]/a")
elem_menu2.click()


Loading took too much time!


In [7]:

title_data = []
title = []

column_index = ['Name', 'ID', 'Status', 'Price', 'Size', 'Date', 'Time_step']

soup = BeautifulSoup(driver.page_source, 'html5lib')
transaction_list = soup.find_all('div', {'tabindex': '-1'})

# transaction_qrray
title_data.append(column_index)

for n in range(15):
    temp = []
    for j in range(int(len(transaction_list)/9-3)):
        if str(transaction_list[10*j+2]).split('<', 10)[2].split('>', 10)[1]=='':
            pass
        else:

            temp.append([str(transaction_list[10*j+2]).split('<', 10)[2].split('>', 10)[1],
                        str(transaction_list[10*j+3]).split('<', 10)[1].split('>', 10)[1],
                        str(transaction_list[10*j+4]).split('<', 10)[1].split('>', 10)[1],
                        str(transaction_list[10*j+7]).split('<', 10)[1].split('>', 10)[1],
                        str(transaction_list[10*j+6]).split('<', 10)[1].split('>', 10)[1].replace(',',''),
                        str(transaction_list[10*j+1]).split('<', 10)[1].split('>', 10)[1],
                        str(transaction_list[10*j+5]).split('<', 10)[1].split('>', 10)[1]])
    
    title_data.append(np.array(temp))
    
    # Scroll the webpage
    for m in range(2):
        try:
            element = driver.find_element_by_xpath('//*[@id="center"]/div/div[4]/div[3]/div/div/div[20]')

            driver.execute_script("return arguments[0].scrollIntoView(true);", element)
            time.sleep(1)
        except:
            pass
    # Reloaging the information of webpage
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    transaction_list = soup.find_all('div', {'tabindex': '-1'})

# Remove the duplicate data
new = []
new.append(title_data[0])
new.append(title_data[1])
for n in range(1,len(title_data)-1):
    for i in range(1,title_data[n+1].shape[0]):
        if np.mean(title_data[n][-i:]==title_data[n+1][:i])==1:
            new.append(title_data[n+1][i:])
            break
# Remake the dataset
final_array =[]
final_array.append(new[0])
for i in range(1,len(new)):
    for j in range(new[i].shape[0]):
        final_array.append(new[i][j])
final_array = np.array(final_array)

In [8]:
# simplify, counting the size
final_array_s = []
final_array_s.append(final_array[0])
count=0
temp_size=0
for i in range(1,final_array.shape[0]):
    try:
        judgment = ((final_array[i][0]==final_array[i+1][0]) and (final_array[i][2]==final_array[i+1][2]) and (final_array[i][-1]==final_array[i+1][-1]))
    except IndexError:
        judgment = False
    if judgment:
        count +=1
        temp_size += int(final_array[i][4].replace(',',''))
    elif count> 0 and ~judgment:
        temp_size += int(final_array[i][4].replace(',',''))
        final_array_s.append([final_array[i][0],final_array[i][1],final_array[i][2],final_array[i][3],str(temp_size),final_array[i][5],final_array[i][6]])
        count =0
        temp_size=0
    else:
        final_array_s.append(final_array[i])
        count =0
        temp_size=0
final_array_s = np.array(final_array_s)

# simplify again, merge the closed time
final_array_ss = []
final_array_ss.append(final_array_s[0])
count=0
for i in range(1,final_array_s.shape[0]):
    try:
        judgment = ((final_array_s[i][0]==final_array_s[i+1][0]) and (final_array_s[i][2]==final_array_s[i+1][2]) and (final_array_s[i][-1][:5]==final_array_s[i+1][-1][:5]))
    except IndexError:
        judgment = False    
    if judgment:
        count +=1
        temp_size += int(final_array_s[i][4].replace(',',''))
    elif count> 0 and ~judgment:
        temp_size += int(final_array_s[i][4].replace(',',''))
        final_array_ss.append([final_array_s[i][0],final_array_s[i][1],final_array_s[i][2],final_array_s[i][3],str(temp_size),final_array_s[i][5],final_array_s[i][6]])
        count =0
        temp_size=0
    else:
        final_array_ss.append(final_array_s[i])
        count =0
        temp_size=0
final_array_ss = np.array(final_array_ss)

In [9]:
# Pre-processing data to yf
Stock_index = []

for i in np.unique(final_array_ss[1:].T[1]):
    Stock_index.append(i)

Index_list = []

for s in range(len(Stock_index)):
    temp = []
    for i in range(final_array_ss.shape[0]-1):
        if Stock_index[s]==final_array_ss[1:].T[1][i]:
            temp.append(final_array_ss[1:][i])
    temp = np.array(temp) 
    Index_list.append(temp)
    
x_timestep_B = []
y_value_B = []  
text_B = []
x_timestep_S = []
y_value_S = []  
text_S = []

for Arr in Index_list:
    temp_xB = []
    temp_yB = []
    temp_tB = []
    temp_xS = []
    temp_yS = []
    temp_tS = []
    for i in range(Arr.shape[0]):
        Temp_Date =  (Arr.T[5])[i].split('/',3)
        Temp_Timestep = (Arr.T[-1][i]).split(':',3)
        Year = int(Temp_Date[0])+1911
        Month = int(Temp_Date[1])
        Day = int(Temp_Date[2])
        Date = str(Year)+ '-'+ str(Month)+ '-'+ str(Day)
        Next_Date = str(Year)+ '-'+ str(Month)+ '-'+ str(Day+1)
        Hour = int(Temp_Timestep[0])
        Min = int(Temp_Timestep[1])
        Sec = int(Temp_Timestep[2])
        
        if Arr.T[2][i]=='現股買':
            temp_xB.append(datetime(Year, Month, Day, Hour, Min, Sec).isoformat())
            temp_yB.append(float(Arr.T[3][i]))
            temp_tB.append(Arr.T[4][i])
        else:
            temp_xS.append(datetime(Year, Month, Day, Hour, Min, Sec).isoformat())
            temp_yS.append(float(Arr.T[3][i]))
            temp_tS.append(Arr.T[4][i])
    x_timestep_B.append(temp_xB)
    y_value_B.append(temp_yB)
    text_B.append(temp_tB)
    x_timestep_S.append(temp_xS)
    y_value_S.append(temp_yS)    
    text_S.append(temp_tS)

In [13]:
# Chose the number form Stocks

# intraday_index(II)
II=0

###
# load the dataset of stock form yahoo

stock_No = Stock_index[II]+'.TW'
start_date = Date
end_date = Next_Date
df = yf.download(tickers = stock_No, start=start_date, end=end_date, period='1d', interval='1m')
if len(df.index)>0:
    pass
else:
    sys.exit("{}: No data found".format(Index_list[II][0][0]))
    
###
# Find the index of candlestick
x_timestep_cd_B = []
x_timestep_cd_S = []
for i in range(len(x_timestep_B)):
    tmep_cd_B = []
    for j in range(len(x_timestep_B[i])):
        dt = (df.index[1]-df.index[0])
        for k in range(len(df.index)):
            Jm = (df.index[k]).ctime()<=datetime.strptime(x_timestep_B[i][j], "%Y-%m-%dT%H:%M:%S").ctime()<(df.index[k]+dt).ctime()
            if Jm:
                tmep_cd_B.append(df.index[k])
                
    tmep_cd_S = []
    for j in range(len(x_timestep_S[i])):                
        for k in range(len(df.index)):
            Jm = (df.index[k]).ctime()<=datetime.strptime(x_timestep_S[i][j], "%Y-%m-%dT%H:%M:%S").ctime()<(df.index[k]+dt).ctime()
            if Jm:
                tmep_cd_S.append(df.index[k])
                
    x_timestep_cd_B.append(tmep_cd_B)
    x_timestep_cd_S.append(tmep_cd_S)
    
###
# plot the figure
fig = go.Figure()

# Candlestick
fig.add_trace(go.Candlestick(x=df.index,
             open=df.Open,
             high=df.High,
             low=df.Low,
             close=df.Close,
             increasing_line_color='#FF6347',
             decreasing_line_color='#90EE90',
             increasing_fillcolor='#FF6347',
             decreasing_fillcolor='#90EE90',
             opacity=1,
             name="<b>{}</b>".format(Index_list[II][0][0])))
    
# Tag Buy and Sell
for b in range(len(x_timestep_cd_B[II])):
    fig.add_trace(go.Scatter(x=[x_timestep_cd_B[II][b]], 
                             y=[y_value_B[II][b]], 
                             mode='markers',
                             marker_symbol='line-ew', 
                             name='Buy'+' '+x_timestep_B[II][b][11:],
                             hovertemplate="<b>Buy</b><br><br>" +
                                            "Prize: {}<br>".format(y_value_B[II][b]) +
                                            "Size: {}<br>".format(text_B[II][b]) +
                                            "Timestep: {}<br>".format(x_timestep_B[II][b].replace('T','  ')) +
                                            "<extra></extra>",
                             marker=dict(color='#FFFF00', 
                                         size=10,
                                         line=dict(color='#0080FF',
                                                   width=3))))
for s in range(len(x_timestep_cd_S[II])):
    fig.add_trace(go.Scatter(x=[x_timestep_cd_S[II][s]], 
                             y=[y_value_S[II][s]], 
                             mode='markers',
                             marker_symbol='line-ew', 
                             name='Sell'+' '+x_timestep_S[II][s][11:],
                             hovertemplate="<b>Sell</b><br><br>" +
                                            "Prize: {}<br>".format(y_value_S[II][s]) +
                                            "Size: {}<br>".format(text_S[II][s]) +
                                            "Timestep: {}<br>".format(x_timestep_S[II][s].replace('T','  ')) +
                                            "<extra></extra>",
                             marker=dict(color='#00BFFF', 
                                         size=10,
                                         line=dict(color='#000000',
                                                   width=3))))

# x-axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15,label='15m',step='minute',stepmode='backward'),
            dict(count=45,label='45m',step='minute',stepmode='backward'),
            dict(count=1,label='1hr',step='hour',stepmode='backward'),
            dict(count=2,label='2hr',step='hour',stepmode='backward'),
            dict(step='all')
        ])
    )
)


fig.update_layout(plot_bgcolor="#FFFFFF",autosize=True, 
                  xaxis = dict(showgrid = True, gridcolor ='#DCDCDC'),
                  yaxis = dict(showgrid = True, gridcolor ='#DCDCDC'))
# Show figure
#fig.update_layout(title='Stock_research', autosize=True)
chart_studio.plotly.plot(fig, filename = 'Stock_research3', auto_open=True)
fig.show()

[*********************100%***********************]  1 of 1 completed
